In [1]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
from zkstats.core import prover_gen_settings, setup, prover_gen_proof, verifier_verify, get_data_commitment_maps

In [4]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
# srs_path = os.path.join('shared/kzg.srs')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
witness_path = os.path.join('prover/witness.json')

=======================  ZK-STATS FLOW =======================

## Step 1
Verifier calls `export_onnx` with dummy data, to generate the onnx model. Dummy data is used to infer the shape of the model

### Output
- `verifier.onnx` file

In [5]:
data_path = os.path.join('data.json')

In [6]:
scales = [2]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

## Step 2
- User defines their computation in a function with signature `computation(state: State, x: list[torch.Tensor])`.
- Prover calls `computation_to_model(computation)` to derive the actual model.
- Prover calls `prover_gen_settings`: export onnx file and compute the settings required by `ezkl.calibrate_settings`

In [7]:
from zkstats.computation import State, computation_to_model


def computation(state: State, x: list[torch.Tensor]):
    out_0 = state.median(x[0])
    out_1 = state.median(x[1])
    return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))

_, prover_model = computation_to_model(computation)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)


/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43530/4262130413.py:7: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_43530/4262130413.py:7: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  return state.mean(torch.tensor([out_0, out_1]).reshape(1,-1,1))
/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the tra

==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-192,338],"logrows":12,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":2624,"total_assignments":302,"total_const_size":46,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2,2],"module_sizes":{"kzg":[],"poseidon":[2624,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}}],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1709450403352}


## Step 3
- Prover generates proof with `prover_gen_proof`:
    - 1. Compile circuit with `compile_circuit`
        - Inputs: onnx model, settings path
        - Outputs: compiled 
    - 2. Generate witness with
- Verifier generates keys with 

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(prover_model_path, prover_compiled_model_path, settings_path, vk_path, pk_path )

print("=======================================")
# Prover generates proof
# print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 0.45028138160705566 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 12.75
==== Generating Proof ====
proof:  {'instances': [['43aef79b2c02e756f80f1c9c237be69f1b7497caef637e58f03450a15b654b21', 'f1a01cfac5286cb0b9c26b8dcd3e1437756456e3e433a5571d64d0781222490e', '0100000000000000000000000000000000000000000000000000000000000000', '3300000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x2c1a90d0c1cad26bbf06847e40f7d806f74630aac569cdf49ac990cfdf2e976d17c3b61ba6db3e2c40845d3dc9c802d300d079031cab38e49a7be7d7dae7b0be1d5113f5ad3b6e339b76f3a6844651853f7df28acd053d1871609e6aac4e603916eca9ffe1b8fe0e1be093737236a333e8e8e719db87a16648b7b806556bc1052fac3d927d5c7d3d883d8f45aa31724be2c1fb34822d82b236dfb7c831c479220132f381eeff7b747831fe2eae3a74c9fee5af6ff669297d1e09672e406c64360f71f25f53c0f24bdd2e74308925f1545350d122a8f396da0ab422694ae948012fb1d9040ec7856b913396b176d86284de29a783d65c6608b9f49bdb17aa5ed91d7e6e0c7

In [9]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

[12.75]